In [6]:
import pandas as pd
import numpy as np
import glob

In [16]:
path = "C:/Users/ozgur/Downloads/fantasy-basketball-master/data/Boxscores/2014-15"
all_files = glob.glob(path + "/*.csv")

In [17]:
temp_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    temp_list.append(df)

dataframe = pd.concat(temp_list, axis=0, ignore_index=True)

In [18]:
print(dataframe)

                   Name      Date Team  Home    W  W_PTS    L  L_PTS     MP  \
0           Monta Ellis  20141028  DAL     0  SAS    101  DAL    100  37.22   
1      Chandler Parsons  20141028  DAL     0  SAS    101  DAL    100  33.82   
2         Dirk Nowitzki  20141028  DAL     0  SAS    101  DAL    100  31.67   
3         Jameer Nelson  20141028  DAL     0  SAS    101  DAL    100  30.47   
4        Tyson Chandler  20141028  DAL     0  SAS    101  DAL    100  27.75   
...                 ...       ...  ...   ...  ...    ...  ...    ...    ...   
32077    Meyers Leonard  20150425  POR     1  MEM    115  POR    109   5.08   
32078        Alonzo Gee  20150425  POR     1  MEM    115  POR    109   0.00   
32079      Allen Crabbe  20150425  POR     1  MEM    115  POR    109   0.00   
32080       Tim Frazier  20150425  POR     1  MEM    115  POR    109   0.00   
32081     Joel Freeland  20150425  POR     1  MEM    115  POR    109   0.00   

       FG  ...  ORB_perc  DRB_perc  TRB_perc  AST_p